In [1]:
import sqlite3
import pandas as pd


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Создаем соединение с базой данных в памяти
con = sqlite3.connect(':memory:')
table_names = [
    'Categories',
    'Customers',
    'Employees',
    'OrderDetails',
    'Orders',
    'Products',
    'Shippers',
    'Suppliers',
]

base_path = 'C:\\Users\\Roadmarshal\\Downloads\\'

# Загружаем все CSV файлы в базу данных
for table in table_names:
    file_path = f'{base_path}{table}.csv'
    data = pd.read_csv(file_path, sep=';')
    data.to_sql(table, con, index=False, if_exists='replace')
# Счетчик заданий
tasks = 0
# Задание 1: Выберите заказчиков из Германии, Франции и Мадрида.
query1 = '''
SELECT CustomerName, Country, Address FROM Customers
WHERE Country IN ('Germany', 'France') OR City = 'Madrid'
'''
result1 = pd.read_sql(query1, con)
print('Задание 1:')
print(result1)
tasks += 1
# Задание 2: Выберите топ 3 страны по количеству заказчиков.
query2 = '''
SELECT Country, COUNT(*) AS CustomerCount
FROM Customers
GROUP BY Country
ORDER BY CustomerCount DESC
LIMIT 3
'''
result2 = pd.read_sql(query2, con)
print('\nЗадание 2:')
print(result2)
tasks += 1
# Задание 3: Выберите перевозчика, который отправил 10-й по времени заказ.
query3 = '''
SELECT Shippers.ShipperName, Orders.OrderDate
FROM Orders
JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID
ORDER BY Orders.OrderDate
LIMIT 1 OFFSET 9
'''
result3 = pd.read_sql(query3, con)
print('\nЗадание 3:')
print(result3)
tasks += 1
# 4 задание, 1 часть: Выбор самого дорогого товара
query4_1 = '''
SELECT ProductName, MAX(Price) AS MaxPrice
FROM Products
'''
result4_1 = pd.read_sql(query4_1, con)
print('\n4 задание, 1 часть: Самый дорогой товар:')
print(result4_1)

# 4 задание, 2 часть: Диапазон цен по категориям с описанием, результат по столбцам
query4_2 = '''
SELECT Categories.CategoryName, MIN(Products.Price) AS MinPrice, MAX(Products.Price) AS MaxPrice, Categories.Description
FROM Categories
LEFT JOIN Products ON Categories.CategoryID = Products.CategoryID
GROUP BY Categories.CategoryID
'''

result4_2 = pd.read_sql(query4_2, con)
print('\n4 задание, 2 часть: Диапазон цен по категориям с описанием:')
print(result4_2)
tasks += 1
# Задание 5: Какой товар больше всего заказывали по количеству единиц товара. Выводим название товара и общее количество единиц.
query_most_popular_product = '''
SELECT Products.ProductID, Products.ProductName, SUM(OrderDetails.Quantity) AS TotalQuantity
FROM Products
JOIN OrderDetails ON Products.ProductID = OrderDetails.ProductID
GROUP BY Products.ProductID
ORDER BY SUM(OrderDetails.Quantity) DESC
LIMIT 1
'''

most_popular_product = pd.read_sql(query_most_popular_product, con)
print('\nСамый популярный товар и общее количество заказанных единиц:')
print(most_popular_product)

# Извлекаем ProductID самого популярного товара
most_popular_product_id = most_popular_product['ProductID'].iloc[0]

# Теперь, зная ProductID, мы можем получить список всех заказов с этим товаром
query_orders_for_most_popular = f'''
SELECT OrderDetails.OrderID, OrderDetails.Quantity
FROM OrderDetails
WHERE OrderDetails.ProductID = {most_popular_product_id}
'''
orders_for_most_popular = pd.read_sql(query_orders_for_most_popular, con)
print('\nВсе заказы для самого популярного товара:')
print(orders_for_most_popular)
tasks += 1
# Задание 6: Топ 5 поставщиков по количеству заказов
query6 = '''
SELECT Suppliers.SupplierName, Suppliers.Country, Suppliers.ContactName, Suppliers.Phone, COUNT(DISTINCT OrderDetails.OrderID) AS QuantityOfOrders
FROM Suppliers
JOIN Products ON Suppliers.SupplierID = Products.SupplierID
JOIN OrderDetails ON Products.ProductID = OrderDetails.ProductID
GROUP BY Suppliers.SupplierID
ORDER BY QuantityOfOrders DESC
LIMIT 5
'''

result6 = pd.read_sql(query6, con)
print('\nЗадание 6: Топ-5 поставщиков по количеству заказов:')
print(result6)
tasks += 1
# 10 задание: Сколько банок крабового мяса всего было заказано.
query10 = '''
SELECT SUM(OrderDetails.Quantity) as TotalCrabMeatCans
FROM Products
JOIN OrderDetails ON Products.ProductID = OrderDetails.ProductID
WHERE Products.ProductName LIKE '%крабовое мясо%' OR
      Products.ProductName LIKE '%мясо крабовое%' OR
      Products.ProductName LIKE '%мясо краба%' OR
      Products.ProductName LIKE '%crab meat%' OR
      Products.ProductName LIKE '%meat crab%'
'''

result10 = pd.read_sql(query10, con)
print('\n10 задание: Общее количество заказанного крабового мяса:')
print(result10)
tasks += 1
# 9 задание: Количество заказов у трех самых молодых сотрудников
query9 = '''
SELECT COUNT(Orders.EmployeeID) as NumberOfOrders,
       Employees.FirstName || ' ' || Employees.LastName as FullName
FROM Employees
JOIN Orders ON Employees.EmployeeID = Orders.EmployeeID
GROUP BY Orders.EmployeeID
ORDER BY DATE(Employees.BirthDate) DESC
LIMIT 3
'''

result9 = pd.read_sql(query9, con)
print('\n9 задание: Количество заказов у трех самых молодых сотрудников и их имена:')
print(result9)
tasks += 1
# 8 задание: Какая разница в стоимости между самым дорогим и самым дешевым заказом из США
query8 = '''
WITH OrderCost AS (
    SELECT 
        Orders.OrderID, 
        SUM(Products.Price * OrderDetails.Quantity) AS TotalCost
    FROM Orders
    JOIN Customers ON Orders.CustomerID = Customers.CustomerID
    JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    WHERE Customers.Country = 'USA'
    GROUP BY Orders.OrderID
)
SELECT 
    MAX(TotalCost) AS MaxCost,
    MIN(TotalCost) AS MinCost,
    MAX(TotalCost) - MIN(TotalCost) AS PriceDifference
FROM OrderCost
'''

result8 = pd.read_sql(query8, con)
print('\n8 задание: Максимальная и минимальная стоимость заказов из США, а также их разница:')
print(result8)
tasks += 1
# 7 задание: Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.
query7 = '''
WITH TotalCosts AS (
    SELECT 
        Categories.CategoryName,
        SUM(Products.Price * OrderDetails.Quantity) AS TotalCategoryCost
    FROM Orders
    JOIN Customers ON Orders.CustomerID = Customers.CustomerID
    JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    JOIN Categories ON Products.CategoryID = Categories.CategoryID
    WHERE Customers.Country = 'Brazil'
    GROUP BY Categories.CategoryName
    ORDER BY TotalCategoryCost DESC
    LIMIT 1
)
SELECT 
    'Brazil' AS Country, 
    CategoryName, 
    TotalCategoryCost
FROM TotalCosts
'''

result7 = pd.read_sql(query7, con)
print('\n7 задание: Категория товаров, заказываемая больше всего по стоимости в Бразилии:')
print(result7)
tasks += 1
if tasks == 10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')


Задание 1:
                            CustomerName  Country  \
0                    Alfreds Futterkiste  Germany   
1                Blauer See Delikatessen  Germany   
2                   Blondel père et fils   France   
3              Bólido Comidas preparadas    Spain   
4                               Bon app'   France   
5              Drachenblut Delikatessend  Germany   
6                        Du monde entier   France   
7   FISSA Fabrica Inter. Salchichas S.A.    Spain   
8                      Folies gourmandes   France   
9                         Frankenversand  Germany   
10                   France restauration   France   
11                       Königlich Essen  Germany   
12                  La corne d'abondance   France   
13                      La maison d'Asie   France   
14                   Lehmanns Marktstand  Germany   
15                Morgenstern Gesundkost  Germany   
16                    Ottilies Käseladen  Germany   
17                     Paris spécia